In [2]:

#Finding Lanes for Self Driving Cars
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [23]:

def ROI(image):
    height=image.shape[0]
    ROI_vertices=np.array([
        [ (200,height),
          (1100,height),
          (550,250)]
    ])
    
    #black image mask of the same size as of the image
    mask=np.zeros_like(image)
    
    #filling the mask in th shape of polygon of ROI with white color
    cv2.fillPoly(mask,ROI_vertices,255)
    
    #cv2.imshow('mask',mask)
    #defining the mask in the original image 
    masked_image=cv2.bitwise_and(image,mask)
    
    return masked_image
    

def show_lines(image,lines):
    #black image mask of the same size as that of image
    line_image=np.zeros_like(image)
    
    #iterate over lines
    if lines is not None:
        for line in lines:
            #converting the 2-D array returned by canny into 1-D array
            x1,y1,x2,y2=line.reshape(4)
            #draw the line on the black mask image
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
            
    return line_image

def coordinates(image,parameters):
    slope,intercept=parameters
    #print(image.shape)
    
    #finding the coordinates
    y1=image.shape[0]
    y2=int(y1*(2/3))
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    
    #return coordinates as 1-D array
    return np.array([x1,y1,x2,y2])


def average_parameters(images,lines):
    #define array for left lane and right lane
    left_fit=[]
    right_fit=[]
    
    #iterate over each line
    for line in lines:
        #reashape line in 1-D
        x1,y1,x2,y2=line.reshape(4)
        
        #getting parameters- slope and intercept of the line which a 1 degree fit
        parameters=np.polyfit(([x1,x2]),([y1,y2]),1)
        
        #print(parameters)
        
        slope=parameters[0]
        intercept=parameters[1]
        
        #check whether the line is of left lane or right lane
        #for left lane slope<0 and for right lane slope>0
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
            
    #print(left_fit)
    #print(right_fit)
    
    #average out these value into a single slope and intercept for one line
    left_fit_average=np.average(left_fit,0)
    right_fit_average=np.average(right_fit,0)
    #print(left_fit_average,'left')
    #print(right_fit_average,'right')
    
    #apart from slope and intercept to draw lines we also need the co-ordinates of the line
    left_line=coordinates(images,left_fit_average)
    right_line=coordinates(images,right_fit_average)
    
    return np.array([left_line,right_line])
    
def process(img):
    #reading the input image
    #img=cv2.imread(r"C:\Users\INDIA\Downloads\opencv-master\opencv-master\samples\data\road.jpg")
    #cv2.imshow('image',img)
    #3plt.imshow(img)
    #plt.show()
    #copy image
    lane_image=np.copy(img)

    #gray conversion
    gray=cv2.cvtColor(lane_image,cv2.COLOR_BGR2GRAY)

    #cv2.imshow('Gray',gray)

    #smoothing the image using Gaussian Blur
    blur=cv2.GaussianBlur(gray,(5,5),0)
    #cv2.imshow('Gaussian Blur',blur)

    #edge detection using Canny edge detection method
    canny_image=cv2.Canny(blur,50,150)
    #cv2.imshow('Edges',canny_image)

    #define the region of interest that contains the road lanes for detection
    cropped_image=ROI(canny_image)
    #cv2.imshow('ROI image',cropped_image)

    #applying Hough Line Transform to draw the lines on the cropped image
    lines=cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=0,maxLineGap=5)

    #display the lines returned by HoughLinesP() in the original image
    line_image=show_lines(lane_image,lines)
    #cv2.imshow('Line Image',line_image)

    #Blend this image with original image to show lanes on original image
    #merge the original image with line image
    merged_image=cv2.addWeighted(lane_image,0.8,line_image,1,1)
    #cv2.imshow('Merged Image',merged_image)

    #optimization
    #show one averged line for a group of line that are together by averaging over its parameters
    averaged_lines=average_parameters(lane_image,lines)

    line_image=show_lines(lane_image,averaged_lines)
    merged_image=cv2.addWeighted(lane_image,0.8,line_image,1,1)
    #cv2.imshow('Image',line_image)
    return merged_image

#read from the video
cap=cv2.VideoCapture(r"C:\Users\INDIA\Downloads\opencv-master\opencv-master\samples\data\test2.mp4")

while cap.isOpened():
    ret,frame=cap.read()
    frame=process(frame)
    cv2.imshow('Frame',frame)
    
    if cv2.waitKey(1)==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RankWarning: Polyfit may be poorly conditioned
C:\Users\INDIA\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: Ran